In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import robohang
os.chdir(os.path.join(robohang.__path__._path[0], "..", ".."))
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import taichi as ti
ti.init(ti.cuda, default_fp=ti.f64, debug=False, fast_math=False, device_memory_GB=4)

[Taichi] version 1.6.0, llvm 15.0.4, commit f1c6fbbd, linux, python 3.8.17
[Taichi] Starting on arch=cuda


[I 08/07/24 15:49:06.304 1466563] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [3]:
import omegaconf
from typing import Literal, List
import tqdm
import numpy as np
import copy
import trimesh
import trimesh.transformations as tra
from robohang.asset.generate_clothes import generate_mesh, export_meta_data, detect_self_intersection
import robohang.common.utils as utils

In [4]:
old_cfg = omegaconf.OmegaConf.load("config/asset/generate_cloth_mesh.yaml")

In [5]:
def get_new_cfg(
    old_cfg: omegaconf.DictConfig,
    clothes_length: Literal["middle", "long", "short"],
    sleeves_length: Literal["long", "short"],
    clothes_width: Literal["thin", "fat"],
    neckline_type: Literal["crewneck", "crewneck_v2", "shirt", "vneck"],
    neckline_size: Literal["middle", "large", "small"],
):
    cfg = copy.deepcopy(old_cfg)

    # body
    if clothes_length == "middle":
        cfg.body.level_z.points = [0.00, 0.11, 0.22, 0.33]
    elif clothes_length == "long":
        cfg.body.level_z.points = [-0.045, 0.08, 0.205, 0.33]
    elif clothes_length == "short":
        cfg.body.level_z.points = [+0.045, 0.14, 0.235, 0.33]
    else:
        raise NotImplementedError(clothes_length)
    
    # sleeve
    if sleeves_length == "long":
        cfg.sleeve.radius_x.points = [0.065, 0.060, 0.055, 0.050]
        cfg.sleeve.radius_x.interp = "quadratic"
        cfg.sleeve.radius_y.points = [0.025, 0.022, 0.020, 0.018]
        cfg.sleeve.radius_y.interp = "quadratic"
        cfg.sleeve.angle_y.points = [90, 120, 120, 120]
        cfg.sleeve.angle_y.interp = "quadratic"
        cfg.sleeve.level_x.points = [0.180, 0.280, 0.380, 0.480]
        cfg.sleeve.level_x.interp = "quadratic"
        cfg.sleeve.level_z.points = [0.425, 0.360, 0.300, 0.240]
        cfg.sleeve.level_z.interp = "quadratic"
        cfg.sleeve.segment = [9, 9, 9]
    elif sleeves_length == "short":
        cfg.sleeve.radius_x.points = [0.065, 0.060]
        cfg.sleeve.radius_x.interp = "linear"
        cfg.sleeve.radius_y.points = [0.025, 0.022]
        cfg.sleeve.radius_y.interp = "linear"
        cfg.sleeve.angle_y.points = [90, 120]
        cfg.sleeve.angle_y.interp = "linear"
        cfg.sleeve.level_x.points = [0.180, 0.280]
        cfg.sleeve.level_x.interp = "linear"
        cfg.sleeve.level_z.points = [0.425, 0.360]
        cfg.sleeve.level_z.interp = "linear"
        cfg.sleeve.segment = [9]
    else:
        raise NotImplementedError(clothes_length)

    # global transform
    if clothes_width == "thin":
        cfg.transform = (
            "tra.scale_matrix(1.2, direction=[0., 0., 1.]) @ "
            "tra.scale_matrix(1.2, direction=[0., 1., 0.]) @ "
            "tra.scale_matrix(1.2, direction=[1., 0., 0.]) @ "
            "tra.translation_matrix([0., -0.52, .1]) @ "
            "tra.rotation_matrix(math.radians(-90), [1., 0., 0.])"
        )
    elif clothes_width == "fat":
        cfg.transform = (
            "tra.scale_matrix(1.2, direction=[0., 0., 1.]) @ "
            "tra.scale_matrix(1.2, direction=[0., 1., 0.]) @ "
            "tra.scale_matrix(1.3, direction=[1., 0., 0.]) @ "
            "tra.translation_matrix([0., -0.52, .1]) @ "
            "tra.rotation_matrix(math.radians(-90), [1., 0., 0.])"
        )
    else:
        raise NotImplementedError(clothes_width)

    # neckline
    if neckline_type == "crewneck":
        if neckline_size == "large":
            cfg.collar.radius_x.points = [0.178, 0.157, 0.113, 0.065]
            cfg.collar.split_xf.points = [0.000, 0.019, 0.038, 0.056]
        elif neckline_size == "middle":
            cfg.collar.radius_x.points = [0.178, 0.155, 0.110, 0.060]
            cfg.collar.split_xf.points = [0.000, 0.018, 0.036, 0.052]
        elif neckline_size == "small":
            cfg.collar.radius_x.points = [0.178, 0.153, 0.107, 0.056]
            cfg.collar.split_xf.points = [0.000, 0.017, 0.034, 0.048]
        else:
            raise NotImplementedError(neckline_size)
    elif neckline_type == "crewneck_v2":
        cfg.collar.angle_xb.points = [0., -4., -12., -24.]
        if neckline_size == "large":
            cfg.collar.radius_x.points = [0.178, 0.157, 0.113, 0.065]
            cfg.collar.split_xf.points = [0.000, 0.000, 0.038, 0.056]
        elif neckline_size == "middle":
            cfg.collar.radius_x.points = [0.178, 0.155, 0.110, 0.060]
            cfg.collar.split_xf.points = [0.000, 0.000, 0.036, 0.052]
        elif neckline_size == "small":
            cfg.collar.radius_x.points = [0.178, 0.153, 0.107, 0.056]
            cfg.collar.split_xf.points = [0.000, 0.000, 0.034, 0.048]
        else:
            raise NotImplementedError(neckline_size)
    elif neckline_type == "shirt":
        cfg.chest.split_x.points = [0.000, 0.000, 0.000, 0.000, 0.005]
        if neckline_size == "large":
            cfg.collar.radius_x.points = [0.178, 0.157, 0.113, 0.065]
            cfg.collar.split_xf.points = [0.005, 0.008, 0.015, 0.056]
        elif neckline_size == "middle":
            cfg.collar.radius_x.points = [0.178, 0.155, 0.110, 0.060]
            cfg.collar.split_xf.points = [0.005, 0.008, 0.015, 0.052]
        elif neckline_size == "small":
            cfg.collar.radius_x.points = [0.178, 0.153, 0.107, 0.056]
            cfg.collar.split_xf.points = [0.005, 0.008, 0.015, 0.048]
        else:
            raise NotImplementedError(neckline_size)
    elif neckline_type == "vneck":
        cfg.chest.split_x.points = [0.000, 0.000, 0.000, 0.000, 0.015]
        if neckline_size == "large":
            cfg.collar.radius_x.points = [0.178, 0.157, 0.113, 0.065]
            cfg.collar.split_xf.points = [0.015, 0.022, 0.039, 0.056]
        elif neckline_size == "middle":
            cfg.collar.radius_x.points = [0.178, 0.155, 0.110, 0.060]
            cfg.collar.split_xf.points = [0.015, 0.022, 0.038, 0.052]
        elif neckline_size == "small":
            cfg.collar.radius_x.points = [0.178, 0.153, 0.107, 0.056]
            cfg.collar.split_xf.points = [0.015, 0.022, 0.037, 0.048]
        else:
            raise NotImplementedError(neckline_size)
    else:
        raise NotImplementedError(neckline_type)
    
    return cfg

In [6]:
def get_all_cfg():
    cfg_list = []
    name_list = []
    clothes_num = 0
    for neckline_type in ["crewneck", "crewneck_v2", "shirt", "vneck", ]:
        for neckline_size in ["middle", "large", "small"]:
            for clothes_width in ["thin", "fat"]:
                for clothes_length in ["middle", "long", "short"]:
                    for sleeves_length in ["long", "short"]:
                        '''if not (
                            clothes_length == "middle" and
                            sleeves_length == "long"
                        ):
                            continue'''
                        cfg_list.append(get_new_cfg(
                            old_cfg, 
                            clothes_length, sleeves_length, clothes_width, 
                            neckline_type, neckline_size,
                        ))
                        name_list.append(dict(
                            neckline_type=neckline_type,
                            neckline_size=neckline_size,
                            clothes_width=clothes_width,
                            clothes_length=clothes_length,
                            sleeves_length=sleeves_length,
                        ))
                        clothes_num += 1

    return cfg_list, name_list, clothes_num

cfg_list, name_list, clothes_num = get_all_cfg()

In [7]:
np.random.seed(0)
permutated = np.random.permutation(clothes_num)
# permutated = np.arange(clothes_num)
train_idx = permutated[:120]
valid_idx = permutated[120:]
print(clothes_num)
print(train_idx)
print(valid_idx)

144
[  7  89  97  26 110 128  59  22 129  16 126 120  40  45  54  33  24   8
 127  51  86 116  37  96  43 114  63 140 101  18  71  27 141   2  60  10
  76 105  56 108  61  44  66 112  95  92  50  30 131  83  98  62 122  90
 113  48 138  73  13 142 100  84  78 124  15  52   3 118 135   6  68  85
  12  91 109  93  46  11 121 104  41 106   1 102 133  42   4 119  17  38
   5  53 137  94   0  34  28  55  75  35  23  74  31 107  57 125  65  32
 132  14 111  19  29  49 130  99  82  64 143  79]
[ 69 123  80 115  20 139  72  77  25  81 134 136  39  58  88  70  87  36
  21   9 103  67 117  47]


In [8]:
def export_all_clothes(
    idx_list: List[int],
    clothes_cls: Literal["train", "valid"],
):
    for export_idx, idx in tqdm.tqdm(enumerate(idx_list), total=len(idx_list)):
        if idx >= len(cfg_list) or idx >= len(name_list):
            continue
        cfg, name = cfg_list[idx], name_list[idx]
        mesh, key_points = generate_mesh(cfg)

        si, intersection_mask = detect_self_intersection(mesh)
        fids, eids = np.where(intersection_mask)

        annotates = []
        self_intersection = False
        for fid in fids:
            self_intersection = True
            for i in range(3):
                xyz = mesh.vertices[mesh.faces[fid, i]]
                annotates.append(trimesh.primitives.Sphere(.01, xyz))
        for eid in eids:
            self_intersection = True
            for i in range(2):
                xyz = mesh.vertices[mesh.edges[eid, i]]
                annotates.append(trimesh.primitives.Box([.008] * 3, tra.translation_matrix(xyz)))
        if self_intersection:
            print(f"self intersection [fid, eid]:\n{fids, eids}")
        
        mesh_file_path = os.path.join("assets/clothes", clothes_cls, utils.format_int(export_idx, len(idx_list) - 1), "clothes.obj")
        os.makedirs(os.path.dirname(mesh_file_path), exist_ok=True)
        trimesh.util.concatenate(annotates + [mesh]).export(mesh_file_path)
        export_meta_data(cfg, key_points, mesh_file_path)
        with open(os.path.join(
            os.path.dirname(mesh_file_path), 
            f"{name['neckline_type']}-{name['neckline_size']}-{name['clothes_width']}-c_{name['clothes_length']}-s_{name['sleeves_length']}"
        ), "w") as f_obj:
            pass

export_all_clothes([0], "train_test")
export_all_clothes(train_idx.tolist(), "train")
export_all_clothes(valid_idx.tolist(), "valid")


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:18<00:00,  1.28it/s]
